In [3]:
# main_evaluation.py

import os
import pandas as pd
from tqdm import tqdm

# NLTK BLEU, METEOR 관련
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

# ROUGE 관련
from rouge_score import rouge_scorer

# Ollama 모델 로드 (사용자 정의)
# ollama_model_load.py 파일에서 임포트하는 객체
from ollama_model_load import DUChatbot5ep
from ollama_model_load import DUChatbot10ep
from ollama_model_load import DUChatbot15ep

# LangChain 메시지 구조
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

# 중간/최종 결과를 저장할 때 사용할 컬럼 헤더
columns = [
    "instruction",
    "reference",
    "generated",
    "bleu_score",
    "meteor_score",
    "rouge_1",
    "rouge_2",
    "rouge_L"
]


def calculate_bleu(reference: str, generated: str) -> float:
    """
    BLEU 점수를 계산한다.
    reference와 generated를 공백 기준으로 나눈 뒤
    NLTK sentence_bleu 함수를 사용한다.
    smoothing_function은 문장 길이에 따른 점수 편차를 줄이기 위해 사용.
    """
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    smoothing = SmoothingFunction().method1
    bleu = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing)
    return bleu


def calculate_meteor(reference: str, generated: str) -> float:
    """
    METEOR 점수를 계산한다.
    meteor_score 함수는 토큰 단위 리스트를 입력으로 받으므로
    문자열을 공백 기준으로 분할한 뒤 리스트로 전달한다.
    """
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    meteor = meteor_score([reference_tokens], generated_tokens)
    return meteor


def calculate_rouge(reference: str, generated: str) -> dict:
    """
    ROUGE-1, ROUGE-2, ROUGE-L 세 가지 점수를 계산한다.
    use_stemmer=True 설정 시 내부적으로 어간 추출을 적용해
    보다 일반화된 평가를 수행한다.
    """
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, generated)

    return {
        "rouge_1": scores["rouge1"].fmeasure,
        "rouge_2": scores["rouge2"].fmeasure,
        "rouge_L": scores["rougeL"].fmeasure
    }


def get_chat_ollama(model_name: str):
    """
    model_name에 따라 ollama_model_load.py에서 임포트해 온
    Ollama 모델 객체를 반환한다.
    """
    if model_name == "DUCChatbot5ep":
        return DUChatbot5ep
    elif model_name == "DUChatbot10ep":
        return DUChatbot10ep
    elif model_name == "DUChatbot15ep":
        return DUChatbot15ep
    else:
        raise ValueError(f"알 수 없는 모델 이름입니다: {model_name}")


def query_llm(chat_ollama_model, prompt_text: str) -> str:
    """
    LangChain의 ChatPromptTemplate과 HumanMessage를 활용해
    Ollama 모델에 프롬프트를 전달하고 답변을 받아온다.
    """
    # 템플릿: 사용자 요청(user_input)에 대해 간략한 답변을 작성.
    PROMPT_TEMPLATE = """
    다음은 사용자 요청입니다:
    {user_input}

    이에 대한 답변을 간략히 작성해 주세요:
    """
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    formatted_prompt = template.format(user_input=prompt_text)
    message = HumanMessage(content=formatted_prompt)

    # Ollama 모델 객체에 메시지 리스트를 전달하고, 응답 텍스트를 받아온다.
    response = chat_ollama_model([message])
    return response.content.strip()


def evaluate_model(
    csv_file: str,
    model_name: str,
    output_file: str,
    batch_size: int = 5
) -> None:
    """
    1. csv_file에서 instruction과 reference 텍스트를 각각 읽어온다.
    2. instruction을 Ollama 모델에 전달하여 답변(generated)을 생성한다.
    3. BLEU, METEOR, ROUGE 점수를 계산한다.
    4. batch_size마다, 중간 결과를 output_file에 CSV로 저장한다.
    """

    # 이미 처리한 행 수를 체크해 중간에서부터 이어서 처리하기 위함.
    processed_count = 0
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file, encoding='utf-8-sig')
        processed_count = len(existing_df)

    # CSV 파일 로드 (인코딩 문제 대비 euc-kr로 재시도)
    try:
        df = pd.read_csv(csv_file, encoding='utf-8-sig')
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file, encoding='euc-kr')

    total_rows = len(df)
    if processed_count >= total_rows:
        print("이미 모든 데이터가 처리되었습니다.")
        return

    # Ollama 모델 로드
    chat_ollama_model = get_chat_ollama(model_name)

    evaluation_results = []

    for idx in tqdm(range(processed_count, total_rows), desc="Evaluating"):
        # CSV의 0번째 컬럼: instruction, 1번째 컬럼: reference
        instruction = df.iloc[idx, 0]
        reference = df.iloc[idx, 1]

        # Ollama 모델로 텍스트 생성
        generated_text = query_llm(chat_ollama_model, instruction)

        # 평가 지표 계산
        bleu_score = calculate_bleu(reference, generated_text)
        meteor_score_value = calculate_meteor(reference, generated_text)
        rouge_scores = calculate_rouge(reference, generated_text)

        # 결과 딕셔너리 형태로 저장
        evaluation_results.append({
            "instruction": instruction,
            "reference": reference,
            "generated": generated_text,
            "bleu_score": bleu_score,
            "meteor_score": meteor_score_value,
            "rouge_1": rouge_scores["rouge_1"],
            "rouge_2": rouge_scores["rouge_2"],
            "rouge_L": rouge_scores["rouge_L"]
        })

        # batch_size마다 중간 결과를 output_file에 append 모드로 저장
        if (len(evaluation_results) % batch_size == 0) or (idx == total_rows - 1):
            partial_df = pd.DataFrame(evaluation_results, columns=columns)

            # 이미 처리된 결과가 있으면 헤더 없이 이어붙이고
            # 처음이면 헤더 포함해서 작성
            if os.path.exists(output_file) and processed_count > 0:
                partial_df.to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8-sig')
            else:
                partial_df.to_csv(output_file, mode='w', index=False, header=True, encoding='utf-8-sig')

            evaluation_results = []
            processed_count = idx + 1

    print(f"평가 완료! 결과는 '{output_file}'에 저장되었습니다.")


if __name__ == "__main__":
    # 1) 5ep 모델
    evaluate_model(
        csv_file="QADataset_new.csv",
        model_name="DUCChatbot5ep",
        output_file="E5_PEFT_NoneRAG_new.csv",
        batch_size=5
    )

    # 2) 10ep 모델
    evaluate_model(
        csv_file="QADataset_new.csv",
        model_name="DUChatbot10ep",
        output_file="E10_PEFT_NoneRAG_new.csv",
        batch_size=5
    )

    # 3) 15ep 모델
    evaluate_model(
        csv_file="QADataset_new.csv",
        model_name="DUChatbot15ep",
        output_file="E15_PEFT_NoneRAG_new.csv",
        batch_size=5
    )

Evaluating:   0%|          | 0/421 [00:00<?, ?it/s]

/tmp/ipykernel_1005338/3695103857.py:113: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_ollama_model([message])
Evaluating: 100%|██████████| 421/421 [1:26:47<00:00, 12.37s/it]  


평가 완료! 결과는 'E5_PEFT_NoneRAG_new.csv'에 저장되었습니다.


Evaluating: 100%|██████████| 421/421 [33:38<00:00,  4.80s/it]  


평가 완료! 결과는 'E10_PEFT_NoneRAG_new.csv'에 저장되었습니다.


Evaluating: 100%|██████████| 421/421 [33:39<00:00,  4.80s/it]    

평가 완료! 결과는 'E15_PEFT_NoneRAG_new.csv'에 저장되었습니다.
